In [17]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import random
import requests
import pandas as pd
import re

## Laden des Datensatzes

Jetzt laden wir unseren Datensatz. Wir gehen davon aus, dass der Datensatz eine CSV-Datei mit Verkaufsdaten enthält.

Fetch Public and public Parking Data from the APIs

In [18]:

# Initialize the geolocator
geolocator = Nominatim(user_agent="parking_city_lookup")

def get_city_from_coordinates(lat, lon):
    """
    Uses reverse geocoding to determine the city based on latitude and longitude.
    """
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, language="en")
        if location:
            address_components = location.raw.get("address", {})
            return address_components.get("city") or address_components.get("town") or address_components.get("village")
    except GeocoderTimedOut:
        return None
    except Exception as e:
        print(f"Reverse geocoding failed for {lat}, {lon}: {e}")
        return None

# Function to split opening and closing times and determine open days
def parse_opening_hours(time_range):
    """
    Parses the opening hours string and returns opening time, closing time, and open days.
    """
    time_mapping = {
        "Weekdays 9AM-11PM": ("09:00", "23:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]),
        "Weekdays 9AM-7PM": ("09:00", "19:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]),
        "Weekdays 6AM-6PM": ("06:00", "18:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]),
        "Weekdays 5AM-7PM": ("05:00", "19:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]),
        "Monday-Sunday 9AM-11PM": ("09:00", "23:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]),
        "Saturday 9AM-12AM": ("09:00", "24:00", ["Saturday", "Sunday"]),
        "Sunday 12AM-22AM": ("00:00", "22:00", ["Saturday", "Sunday"])
        
    }
    return time_mapping[time_range]

selected_time_range = random.choice([
    "Weekdays 9AM-11PM", "Weekdays 9AM-7PM", "Weekdays 6AM-6PM", "Weekdays 5AM-7PM", "Monday-Sunday 9AM-11PM", "Saturday 9AM-12AM", "Sunday 12AM-22AM"
])

def get_city_from_address(address):
    if address:
        parts = address.split(", ")
        
        # Check if the last part contains numbers (postal code)
        last_part = parts[-1]
        
        # If the last part contains numbers (assumed to be postal code)
        if re.search(r'\d', last_part):
            # Take the last word (city name)
            city = last_part.split()[-1]
        else:
            # Otherwise, take everything after the last comma as the city
            city = last_part
        
        return city if city else None
    return None


In [19]:


# API URL for Freiburg
api_url_freiburg = "https://api.parkendd.de/Freiburg"

# Fetch the data from the WFS API
response_freiburg = requests.get(api_url_freiburg)

if response_freiburg.status_code == 200:
    public_parking_data = response_freiburg.json()

    # Extract relevant data
    parking_data_public = []
    for site in public_parking_data['lots']:
        if site.get("purpose") and site.get("purpose") != "CAR":
            continue
        
        coords = site.get("coords", {})
        latitude, longitude = coords.get("lat"), coords.get("lng")

        opening_time, closing_time, open_days = parse_opening_hours(selected_time_range)

        parking_data_public.append({
            "public_id": site.get("id"),
            "name": site.get("name"),
            "address": site.get("address"),
            "city": "Freiburg im Breisgau", # Since data is from Freiburg, to reduce time, city is always Freiburg im Breisgau
            "latitude": latitude,
            "longitude": longitude,
            "price_per_hour": random.choice([3.80, 3.20, 1.60]),
            "capacity": site.get("total"),
            "available_space": site.get("free"),
            "opening_time": opening_time,
            "closing_time": closing_time,
            "open_days": open_days  # Open days as an array
        })
    
    # Convert to DataFrame
    df_freiburg = pd.DataFrame(parking_data_public)
    print("Public Parking Data (Freiburg):", df_freiburg.head())
else:
    print("Failed to fetch Freiburg parking data.")



Public Parking Data (Freiburg):                   public_id              name address                  city  \
0         freiburgambahnhof        Am Bahnhof      P4  Freiburg im Breisgau   
1          freiburgkarlsbau          Karlsbau     P13  Freiburg im Breisgau   
2  freiburgzurunterfuehrung  Zur Unterführung      P5  Freiburg im Breisgau   
3     freiburgzaehringertor     Zähringer Tor     P12  Freiburg im Breisgau   
4       freiburgschlossberg       Schlossberg     P15  Freiburg im Breisgau   

    latitude  longitude  price_per_hour  capacity  available_space  \
0  47.998971   7.842917             3.2       175               88   
1  47.997446   7.853848             1.6       656              203   
2  48.001286   7.844732             3.8       100               38   
3  47.998958   7.854022             1.6       162               81   
4  47.994017   7.855142             1.6       430              122   

  opening_time closing_time           open_days  
0        00:00        

In [20]:
# API URL for MobiData BW API (public parking data)
api_url_gebündelte_parkplätze_parkbauten = "https://api.mobidata-bw.de/park-api/api/public/v3/parking-sites"

# Fetch data from MobiData BW API
response_gebündelte_parkplätze_parkbauten = requests.get(api_url_gebündelte_parkplätze_parkbauten)

# Check the response and parse the JSON data
if response_gebündelte_parkplätze_parkbauten.status_code == 200:
    public_parking_data = response_gebündelte_parkplätze_parkbauten.json()
    
    # Extract relevant data (modify based on actual response structure)
    parking_data_public = []
    for site in public_parking_data['items']:
        if site.get("purpose") and site.get("purpose") != "CAR":
            continue

        latitude = float(site.get("lat", 0)) 
        longitude = float(site.get("lon", 0))  
        
        if not (47.52 <= latitude <= 49.79 and 7.50 <= longitude <= 10.49):
            continue
        
        opening_time, closing_time, open_days = parse_opening_hours(selected_time_range)

        address = site.get("address")
        city = get_city_from_address(address) if address else None

        if not city:  # If city is still missing, use geopy reverse geocoding
            city = get_city_from_coordinates(latitude, longitude)

        # Standardize "Freiburg" to "Freiburg im Breisgau"  
        if city == "Freiburg":
            city = "Freiburg im Breisgau"

        parking_data_public.append({
            "public_id": site.get("id"),
            "name": site.get("name"),
            "address": address,
            "city": city,  # Extracted or geocoded city
            "latitude": latitude,
            "longitude": longitude,
            "price_per_hour": random.choice([3.80, 3.20, 1.60]),
            "capacity": site.get("capacity"),
            "available_space": site.get("realtime_free_capacity"),
            "opening_time": opening_time,
            "closing_time": closing_time,
            "open_days": open_days  # Open days as an array
        })
    
    # Convert to DataFrame
    df_public_parking = pd.DataFrame(parking_data_public)
    print("Public Parking Data (MobiData BW):", df_public_parking.head())
else:
    print("Failed to fetch MobiData BW parking data.")


Reverse geocoding failed for 48.7090386, 9.5846259: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=48.7090386&lon=9.5846259&format=json&accept-language=en&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Reverse geocoding failed for 49.0104066, 9.1519612: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=49.0104066&lon=9.1519612&format=json&accept-language=en&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Reverse geocoding failed for 48.8071556, 9.5249867: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=48.8071556&lon=9.5249867&format=json&accept-language=en&addressdetails=1 (Caused by ReadTimeoutError(

In [21]:
df_freiburg

,public_id,name,address,city,latitude,longitude,price_per_hour,capacity,available_space,opening_time,closing_time,open_days
0,freiburgambahnhof,Am Bahnhof,P4,Freiburg im Breisgau,47.998971,7.842917,3.2,175,88,00:00,22:00,"[Saturday, Sunday]"
1,freiburgkarlsbau,Karlsbau,P13,Freiburg im Breisgau,47.997446,7.853848,1.6,656,203,00:00,22:00,"[Saturday, Sunday]"
2,freiburgzurunterfuehrung,Zur Unterführung,P5,Freiburg im Breisgau,48.001286,7.844732,3.8,100,38,00:00,22:00,"[Saturday, Sunday]"
3,freiburgzaehringertor,Zähringer Tor,P12,Freiburg im Breisgau,47.998958,7.854022,1.6,162,81,00:00,22:00,"[Saturday, Sunday]"
4,freiburgschlossberg,Schlossberg,P15,Freiburg im Breisgau,47.994017,7.855142,1.6,430,122,00:00,22:00,"[Saturday, Sunday]"
5,freiburgpaedhochschule,Päd. Hochschule,P20,Freiburg im Breisgau,47.982361,7.890891,1.6,165,25,00:00,22:00,"[Saturday, Sunday]"
6,freiburgwestarkaden,Westarkaden,P22,Freiburg im Breisgau,48.006420,7.827394,3.2,750,532,00:00,22:00,"[Saturday, Sunday]"
7,freiburgschwarzwaldcity,Schwarzwald City,P9,Freiburg im Breisgau,47.997206,7.851119,3.2,391,147,00:00,22:00,"[Saturday, Sunday]"
8,freiburgbahnhofsgarage,Bahnhofsgarage,P1,Freiburg im Breisgau,47.996932,7.841420,1.6,0,0,00:00,22:00,"[Saturday, Sunday]"
9,freiburgvolksbank,Volksbank,P3,Freiburg im Breisgau,47.998021,7.843413,3.2,186,71,00:00,22:00,"[Saturday, Sunday]"


In [22]:
unique_addresses = df_freiburg["address"].unique()
print(unique_addresses)


['P4' 'P13' 'P5' 'P12' 'P15' 'P20' 'P22' 'P9' 'P1' 'P3' 'P16' 'P8' 'P18'
 'P10' 'P21' 'P2' 'P6' 'P19' 'P14']


In [23]:
df_public_parking

,public_id,name,address,city,latitude,longitude,price_per_hour,capacity,available_space,opening_time,closing_time,open_days
0,887,Listplatz 1 / Bahnhofstraße,"Listplatz 1 / Bahnhofstraße, Reutlingen",Reutlingen,48.495983,9.210331,3.8,198.0,NaN,00:00,22:00,"[Saturday, Sunday]"
1,888,Obere Wässere 3 - 7,"Obere Wässere 3 - 7, Reutlingen",Reutlingen,48.488955,9.216701,3.8,181.0,NaN,00:00,22:00,"[Saturday, Sunday]"
2,17161,Parkplatz,Stuttgart,Stuttgart,48.717974,9.107351,3.8,6.0,NaN,00:00,22:00,"[Saturday, Sunday]"
3,17189,Parkplatz,None,Stuttgart,48.801826,9.217801,1.6,16.0,NaN,00:00,22:00,"[Saturday, Sunday]"
4,17813,Parkplatz,Ellwangen (Jagst),Ellwangen (Jagst),48.964766,10.129933,3.8,58.0,NaN,00:00,22:00,"[Saturday, Sunday]"
...,...,...,...,...,...,...,...,...,...,...,...,...
3306,18621,Parkplatz,Remchingen,Remchingen,48.953570,8.573547,3.2,37.0,NaN,00:00,22:00,"[Saturday, Sunday]"
3307,18622,Parkplatz,Remchingen,Remchingen,48.953511,8.572800,3.8,6.0,NaN,00:00,22:00,"[Saturday, Sunday]"
3308,18721,Parkplatz,"Sulzbach (Kr FDS), Loßburg",Loßburg,48.424065,8.451453,3.8,11.0,NaN,00:00,22:00,"[Saturday, Sunday]"
3309,18882,Parkplatz,Endingen (Baden),Endingen (Baden),48.144057,7.702840,3.2,8.0,NaN,00:00,22:00,"[Saturday, Sunday]"


Data Cleaning

In [24]:
# Clean the Freiburg parking data
df_freiburg_cleaned = df_freiburg.dropna(subset=["public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity"])  

# Clean the public parking data
df_public_parking_cleaned = df_public_parking.dropna(subset=["public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity"]) 

# Function to fill NaN in 'available_space' with half of 'capacity'
def fill_available_space_with_half_capacity(df):
    df['available_space'] = df['available_space'].fillna(df['capacity'] / 2)
    df['available_space'] = df['available_space'].astype(int)  # Ensure integer values
    return df

# Apply the function to both datasets
df_freiburg_cleaned = fill_available_space_with_half_capacity(df_freiburg_cleaned)
df_public_parking_cleaned = fill_available_space_with_half_capacity(df_public_parking_cleaned)

# Remove duplicates based on parking ID
df_freiburg_cleaned = df_freiburg_cleaned.drop_duplicates(subset=["public_id"])
df_public_parking_cleaned = df_public_parking_cleaned.drop_duplicates(subset=["public_id"])

df_public_parking_cleaned


C:\Users\vae.tiolamon\AppData\Local\Temp\ipykernel_17960\4106914945.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['available_space'] = df['available_space'].fillna(df['capacity'] / 2)
C:\Users\vae.tiolamon\AppData\Local\Temp\ipykernel_17960\4106914945.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['available_space'] = df['available_space'].astype(int)  # Ensure integer values


,public_id,name,address,city,latitude,longitude,price_per_hour,capacity,available_space,opening_time,closing_time,open_days
0,887,Listplatz 1 / Bahnhofstraße,"Listplatz 1 / Bahnhofstraße, Reutlingen",Reutlingen,48.495983,9.210331,3.8,198.0,99,00:00,22:00,"[Saturday, Sunday]"
1,888,Obere Wässere 3 - 7,"Obere Wässere 3 - 7, Reutlingen",Reutlingen,48.488955,9.216701,3.8,181.0,90,00:00,22:00,"[Saturday, Sunday]"
2,17161,Parkplatz,Stuttgart,Stuttgart,48.717974,9.107351,3.8,6.0,3,00:00,22:00,"[Saturday, Sunday]"
4,17813,Parkplatz,Ellwangen (Jagst),Ellwangen (Jagst),48.964766,10.129933,3.8,58.0,29,00:00,22:00,"[Saturday, Sunday]"
5,757,Bahnhofstraße,"Bahnhofstraße, Reutlingen",Reutlingen,48.497575,9.211077,3.8,128.0,64,00:00,22:00,"[Saturday, Sunday]"
...,...,...,...,...,...,...,...,...,...,...,...,...
3306,18621,Parkplatz,Remchingen,Remchingen,48.953570,8.573547,3.2,37.0,18,00:00,22:00,"[Saturday, Sunday]"
3307,18622,Parkplatz,Remchingen,Remchingen,48.953511,8.572800,3.8,6.0,3,00:00,22:00,"[Saturday, Sunday]"
3308,18721,Parkplatz,"Sulzbach (Kr FDS), Loßburg",Loßburg,48.424065,8.451453,3.8,11.0,5,00:00,22:00,"[Saturday, Sunday]"
3309,18882,Parkplatz,Endingen (Baden),Endingen (Baden),48.144057,7.702840,3.2,8.0,4,00:00,22:00,"[Saturday, Sunday]"


Data Integration

In [25]:
# Ensure latitude and longitude columns are numeric and consistent
df_freiburg_cleaned["latitude"] = pd.to_numeric(df_freiburg_cleaned["latitude"], errors="coerce")
df_freiburg_cleaned["longitude"] = pd.to_numeric(df_freiburg_cleaned["longitude"], errors="coerce")

df_public_parking_cleaned["latitude"] = pd.to_numeric(df_public_parking_cleaned["latitude"], errors="coerce")
df_public_parking_cleaned["longitude"] = pd.to_numeric(df_public_parking_cleaned["longitude"], errors="coerce")

# Drop rows with NaN in latitude or longitude
df_freiburg_cleaned = df_freiburg_cleaned.dropna(subset=["latitude", "longitude"])
df_public_parking_cleaned = df_public_parking_cleaned.dropna(subset=["latitude", "longitude"])

# Round latitude and longitude to the same precision for matching
df_freiburg_cleaned["latitude"] = df_freiburg_cleaned["latitude"].round(6)
df_freiburg_cleaned["longitude"] = df_freiburg_cleaned["longitude"].round(6)

df_public_parking_cleaned["latitude"] = df_public_parking_cleaned["latitude"].round(6)
df_public_parking_cleaned["longitude"] = df_public_parking_cleaned["longitude"].round(6)

# Merge the DataFrames, prioritizing df_public_parking_cleaned data
df_combined = pd.merge(
    df_freiburg_cleaned,
    df_public_parking_cleaned,
    on=["latitude", "longitude"],
    how="outer",
    suffixes=("_freiburg", "_public")
)

In [26]:
# Prioritize data from df_public_parking_cleaned
for column in df_public_parking_cleaned.columns:
    if column not in ["latitude", "longitude"]:  # Exclude keys used for merging
        df_combined[column] = df_combined[column + "_public"].combine_first(df_combined[column + "_freiburg"])

# Drop redundant columns
columns_to_drop = [col for col in df_combined.columns if col.endswith("_freiburg") or col.endswith("_public")]
df_combined = df_combined.drop(columns=columns_to_drop)

df_combined

,latitude,longitude,public_id,name,address,city,price_per_hour,capacity,available_space,opening_time,closing_time,open_days
0,47.998971,7.842917,freiburgambahnhof,Am Bahnhof,P4,Freiburg im Breisgau,3.2,175.0,88.0,00:00,22:00,"[Saturday, Sunday]"
1,47.997446,7.853848,903.0,P13 Karlsbau,"Auf der Zinnen 1, 79098 Freiburg",Freiburg im Breisgau,3.8,977.0,216.0,00:00,22:00,"[Saturday, Sunday]"
2,48.001286,7.844732,919.0,P5 Zur Unterführung,"Stefan-Meier-Straße 19-21, 79104 Freiburg",Freiburg im Breisgau,3.8,100.0,38.0,00:00,22:00,"[Saturday, Sunday]"
3,47.998958,7.854022,917.0,P12 Zähringer Tor,P12,P12,1.6,162.0,81.0,00:00,22:00,"[Saturday, Sunday]"
4,47.994017,7.855142,911.0,P15 Schlossberg,"Schlossbergring 14, 79098 Freiburg",Freiburg im Breisgau,3.2,430.0,118.0,00:00,22:00,"[Saturday, Sunday]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2486,49.437864,8.572040,18393.0,Parkplatz,Friedrichsfeld (MA),Friedrichsfeld (MA),3.2,19.0,9.0,00:00,22:00,"[Saturday, Sunday]"
2487,48.953570,8.573547,18621.0,Parkplatz,Remchingen,Remchingen,3.2,37.0,18.0,00:00,22:00,"[Saturday, Sunday]"
2488,48.953511,8.572800,18622.0,Parkplatz,Remchingen,Remchingen,3.8,6.0,3.0,00:00,22:00,"[Saturday, Sunday]"
2489,48.144057,7.702840,18882.0,Parkplatz,Endingen (Baden),Endingen (Baden),3.2,8.0,4.0,00:00,22:00,"[Saturday, Sunday]"


In [27]:
# Define desired column order
desired_columns = [
    "public_id", 
    # "type",
    "name", 
    "address",
    "city", 
    "latitude", 
    "longitude", 
    "price_per_hour", 
    "capacity", 
    "available_space", 
    "opening_time",
    "closing_time",
    "open_days"
]

# Add the "type" column before reordering
# df_combined["type"] = "public"

# Reorder the columns in the combined DataFrame
df_combined = df_combined[[col for col in desired_columns if col in df_combined.columns]]

df_combined['public_id'] = df_combined['public_id'].apply(lambda x: f"PUB{random.randint(10000, 99999)}")

df_combined

,public_id,name,address,city,latitude,longitude,price_per_hour,capacity,available_space,opening_time,closing_time,open_days
0,PUB17359,Am Bahnhof,P4,Freiburg im Breisgau,47.998971,7.842917,3.2,175.0,88.0,00:00,22:00,"[Saturday, Sunday]"
1,PUB42776,P13 Karlsbau,"Auf der Zinnen 1, 79098 Freiburg",Freiburg im Breisgau,47.997446,7.853848,3.8,977.0,216.0,00:00,22:00,"[Saturday, Sunday]"
2,PUB45096,P5 Zur Unterführung,"Stefan-Meier-Straße 19-21, 79104 Freiburg",Freiburg im Breisgau,48.001286,7.844732,3.8,100.0,38.0,00:00,22:00,"[Saturday, Sunday]"
3,PUB74933,P12 Zähringer Tor,P12,P12,47.998958,7.854022,1.6,162.0,81.0,00:00,22:00,"[Saturday, Sunday]"
4,PUB73006,P15 Schlossberg,"Schlossbergring 14, 79098 Freiburg",Freiburg im Breisgau,47.994017,7.855142,3.2,430.0,118.0,00:00,22:00,"[Saturday, Sunday]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2486,PUB98481,Parkplatz,Friedrichsfeld (MA),Friedrichsfeld (MA),49.437864,8.572040,3.2,19.0,9.0,00:00,22:00,"[Saturday, Sunday]"
2487,PUB91139,Parkplatz,Remchingen,Remchingen,48.953570,8.573547,3.2,37.0,18.0,00:00,22:00,"[Saturday, Sunday]"
2488,PUB66182,Parkplatz,Remchingen,Remchingen,48.953511,8.572800,3.8,6.0,3.0,00:00,22:00,"[Saturday, Sunday]"
2489,PUB69233,Parkplatz,Endingen (Baden),Endingen (Baden),48.144057,7.702840,3.2,8.0,4.0,00:00,22:00,"[Saturday, Sunday]"


In [28]:
df_combined.to_json('public_parking.json', orient='records', indent=4)

print("DataFrame has been saved to 'public_parking.json'")

DataFrame has been saved to 'public_parking.json'
